In [45]:
import pandas as pd 
import process
import numpy as np 
# Jerome path : r'C:\Users\33640\OneDrive\Documents\GitHub\Portfolio_clustering_project\Data\DataBase.csv'
# Nail path : '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv'
df = pd.read_csv(r'/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DataBase.csv')

df.set_index('ticker', inplace=True)

df.columns = pd.to_datetime(df.columns.str[1:], format='%Y%m%d').strftime('%d/%m/%Y')

df_cleaned = df.fillna(0) # Utilisez la méthode fillna(0) pour remplacer les NaN par 0

df_cleaned = df_cleaned.transpose() ## WE WANT COLUMNS TO BE VECTOR OF RETURN FOR A GIVEN TICKER

In [79]:
from sklearn.model_selection import ShuffleSplit

def auxilary_matrix(days, beta, df_cleaned):

    ## 1. We extract the data corresponding to the returns of our assets (columns) during these d days (lines)
    X = df_cleaned.iloc[0:days,:]

    ## 2. We slightly adjust the matrix of observations to get the auxiliary matrix that puts more weight on recent dates

    W = np.sqrt(np.diag(days * (1 - beta) * beta**(np.arange(days)[::-1]) / (1 - beta**days)))  # Compute the weight matrix
    X_tilde = pd.DataFrame(index=X.index, columns=X.columns, data=np.dot(W, X))

    ## 3. We randomize the auxiliary matrix of observations according to the time axis
    Randomized_X = X_tilde.transpose().sample(frac=1, axis=1, random_state=42) ## we transpose X as we want to have daily observations of the whole dataset !

    return Randomized_X

def shuffle_split(array, K):
    # Initialize ShuffleSplit
    shuffle_split = ShuffleSplit(n_splits=K, test_size=0.2, random_state=42) 
    # test_size=0.2 : 20% des données pour l'ensemble de test, 80% pour l'ensemble d'entraînement.

    # Create empty list to store splits
    splits = []

    # Perform shuffling and splitting
    for train_index, test_index in shuffle_split.split(array):
        train_fold = [array[i] for i in train_index]
        test_fold = [array[i] for i in test_index]
        splits.append((train_fold, test_fold)) ## attention à cette structure

    return splits


def eigen_sample(data, train_fold): ## we train the data on this test fold

    train_data = data.loc[:, train_fold]

    # Calculer la moyenne de l'ensemble d'entraînement
    mean_train = np.mean(train_data, axis=1)

    # Centrer les données d'entraînement
    centered_train_data = train_data.sub(mean_train, axis=0)

    # Calculer la matrice de covariance des données d'entraînement
    cov_matrix_train = np.cov(centered_train_data) ## size number of assets * number of assets

    # Calculer les vecteurs et valeurs propres de la matrice de covariance
    _, eigenvectors_train = np.linalg.eig(cov_matrix_train)

    return eigenvectors_train

def intra_fold_loss(data, test_fold): ## we test the data on this test fold
    return

In [81]:
X_tilde = auxilary_matrix(days=250, beta=0.99, df_cleaned=df_cleaned)
array = X_tilde.columns
K = 10 
splits = shuffle_split(array=array, K=K)
train, test = splits[0][0], splits[0][1]
eigenvectors_train = eigen_sample(data=X_tilde, train_fold=train)
eigenvectors_train.shape

(695, 695)

In [76]:
train_data = X_tilde.loc[:, train]
cov = np.cov(train_data)
cov.shape

(695, 695)